In [1]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time
import csv
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [13]:
path=Service(r'C:\Users\f_ati\OneDrive\Documents\Master\Master2\TextMining\Projet\chromedriver_win32\chromedriver.exe')

#ouvrir la page 
driver=webdriver.Chrome(service=path)
url = 'https://www.google.com/maps/place/Disneyland+Paris/@48.8673892,2.7791083,17z/data=!4m7!3m6!1s0x47e61d19ca7ae2bd:0x57faf8cb6310e660!8m2!3d48.8673858!4d2.783593!9m1!1b1'

#accepter les cookies
driver.get(url)
valide=driver.find_element(By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span')
valide.click()
time.sleep(2)

In [26]:
#scroll
total_number_of_reviews = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div/div[2]/div[2]').text.split(" ")[0]
total_number_of_reviews = total_number_of_reviews.replace(u"\u202f","") if u"\u202f" in total_number_of_reviews else int(total_number_of_reviews)
total_number_of_reviews=int(total_number_of_reviews)
scrollable_div = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
#Scroll as many times as necessary to load all reviews
for i in range(0,round(total_number_of_reviews / 1000)):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight',scrollable_div)
    time.sleep(0.3)

In [27]:

response = BeautifulSoup(driver.page_source, 'html.parser')
r = response.find_all('div', class_="jftiEf fontBodyMedium")


rev_dict = {'Review Text': [],
            'Review Name' : [],
            'Review Time': [],
            'Review Role' : [],
            'Review Visite' : [],
            'Review Rate' : []}

for rv in r :
    review_text = rv.find_all('span', class_='wiI7pd')
    review_name = rv.find_all('span', jstcache='335')
    review_date = rv.find_all('span',class_='rsqaWe')
    review_role = rv.find_all('span', jstcache='337') 
    #review_perso_num_rate = rv.find_all('span', jstcache='324') 
    review_visite = rv.find_all('span', jstcache='202')
    review_rate = rv.find_all('span', class_="kvMYJc")
    
    if len(review_text) == 0 : 
        rev_dict['Review Text'].append(" ")
    else :
        for rv in review_text :
            rev_dict['Review Text'].append(rv.get_text().replace(",", " "))
            
    if len(review_name) == 0 :
        rev_dict['Review Name'].append(" ")
    else:
        for rv in review_name : 
            rev_dict['Review Name'].append(rv.get_text())
    if len(review_date) == 0:
        rev_dict['Review Time'].append(" ")
    else:
        for rv in review_date :
            rev_dict['Review Time'].append(rv.get_text())
        
    if len(review_role) == 0 : 
        rev_dict['Review Role'].append(" ")
    else :
        for rv in review_role : 
            rev_dict['Review Role'].append(rv.get_text())
    if len(review_visite) == 0 : 
        rev_dict['Review Visite'].append(" ")
    else :
        for rv in review_visite : 
            rev_dict['Review Visite'].append(rv.get_text())
            
   
    for rv in review_rate : 
        attributes_dictionary = rv.attrs
        stars = attributes_dictionary['aria-label']
        rev_dict['Review Rate'].append(stars)

df=pd.DataFrame(rev_dict)
df.to_csv("DisneylandParc_google_maps.csv")


In [28]:
rev_dict

{'Review Text': ['Au vu du nombre d’avis je vais faire court : Magnifique. Féerique. Grandiose. Mais quelle attente. 90min en moyenne par attraction. Je n’ose même pas imaginer aux périodes de fortes affluences. Et puis le prix. Hôtel + 3 jours afin de …',
  'Les prix ont tellement augmenté en 8 ans  c’est assez dingue.\nLe parc est propre  les mesures Covid-19 bien faite. Toujours des pannes mais ça reste de la mécanique/électronique qui tourne beaucoup donc c’est normal.\nCertaines attractions refaite comme la maison hantée  je n’ai pas vu de différence',
  "Magique  quel que soit l'âge.\nConnu tout petit  vingt ans plus tard ça marche encore. Tout les employés sont adorables  les attractions marchent toujours aussi bien et l'ambiance également. …",
  'Nous sommes allé a Disney un jeudi hors vacances scolaires. Le parc nous a fait retomber en enfance : les personnages de nos dessins animés préférés étaient au rendez-vous. Nous avons choisi la période de la Halloween pour profiter de 